In [ ]:
!pip install -q langchain faiss-cpu datasets transformers sentence-transformers accelerate bitsandbytes einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.3 MB/s eta 0:00:00


In [ ]:
!pip install -U -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install -q pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 28.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

PINECONE_API_KEY = os.getenv(userdata.get('PINECONE'))
PINECONE_ENV = "us-east-1-aws"  # Use this for free tier
INDEX_NAME = "qwen-typhoon-demo"

NameError: name 'os' is not defined

In [ ]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(name=INDEX_NAME, dimension=1024)  # Adjust if Qwen returns different dim

In [ ]:
from langchain.llms.base import LLM
from typing import List, Optional
from openai import OpenAI
import os

class TyphoonAPI(LLM):
    model: str = "typhoon-v2-70b-instruct"
    api_key: Optional[str] = os.getenv("TYPHOON_API_KEY")
    base_url: str = "https://api.opentyphoon.ai/v1"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs) -> str:
        client = OpenAI(api_key=self.api_key, base_url=self.base_url)
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content

    @property
    def _llm_type(self) -> str:
        return "typhoon-api"


In [ ]:
import pandas as pd
df = pd.read_json("hf://datasets/Josephgflowers/Finance-Instruct-500k/train.json", lines=True)
df = df.head(50)

In [ ]:
from langchain.docstore.document import Document

docs = [
    Document(page_content=f"User: {row['user']}\nAssistant: {row['assistant']}")
    for _, row in df.iterrows()
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
chunks = splitter.split_documents(docs)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B" ,
                                  model_kwargs={"device": "cuda"},  # Use GPU if available
                                  encode_kwargs={"batch_size": 64, "normalize_embeddings": True})


/tmp/ipython-input-5-969625418.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")


In [ ]:
from langchain.vectorstores import FAISS

# Function to batch-process and add to FAISS
def build_faiss_vectorstore_batched(docs, batch_size=500):
    all_vectorstore = None
    for i in tqdm(range(0, len(docs), batch_size), desc="Building FAISS"):
        chunk = docs[i:i+batch_size]
        if all_vectorstore is None:
            all_vectorstore = FAISS.from_documents(chunk, embedding_model)
        else:
            sub_store = FAISS.from_documents(chunk, embedding_model)
            all_vectorstore.merge_from(sub_store)
    return all_vectorstore

vectorstore = build_faiss_vectorstore_batched(chunks)

In [ ]:
from langchain.chains import RetrievalQA
from google.colab import userdata

llm = TyphoonAPI(api_key=userdata.get('TYPHOON_API_KEY'))

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)


In [ ]:
query = "What is inflation and how does it affect daily life?"
result = rag_chain(query)

print("Answer:\n", result["result"])
print("\nSources:\n")
for doc in result["source_documents"]:
    print("-", doc.page_content[:200], "\n")


Answer:
 Inflation is the increase in the general price level of goods and services in an economy over time, which leads to a decrease in the purchasing power of money. In daily life, inflation affects individuals and businesses by causing higher prices for goods and services, reducing the value of savings and investments, and potentially leading to slower economic growth. This can impact people's ability to afford necessities and discretionary items, as well as businesses' profitability and decision-making processes. Central banks aim to keep inflation low and stable, around 2%, to ensure economic stability and allow for accurate economic planning.

Sources:

- Hope this overview helps explain the concept of inflation in economics! Let me know if you have any other questions. 

- In summary, inflation causes higher prices that reduce purchasing power, squeeze business profits, hurt the value of investments, and tend to lead to slower economic activity and growth. Maintaining s 

- • C

# Finance Finetuned

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [1]:
PROMPTS_SYS = """ You are a CFA (chartered financial analyst) taking a test to
 evaluate your knowledge of finance.
 You will be given a question along
 with five possible answers (A, B, C, D, Fall, Rise).
 Before answering, you should repeat the question carefully,
 then you should think
 through the question step-by-step.
 Explain your reasoning at each step
 towards answering the question. If
 calculation is required, do each
 step of the calculation as a step
 in your reasoning.
 Indicate the correct answer (A, B, C, D, Fall, Rise).
 Do not think or answer things other than what is asked
 <output>
 "answer": [Reflect and verify the final answer (A , B , C , D & Fall or Rise )]
 </output>
"""

In [25]:
PROMPTS_SYS = """You are a financial analyst taking a test to evaluate your knowledge of finance of different topics in finance. You think step by step approach with reflection to answer queries.

Follow these steps:
1. Think through the problem step by step reflect and verify while reasoning within the <thinking> tags.
2. Please and put the answer your final, concise answer within the <output> tags.

The <thinking> sections are for your internal reasoning process only.
Do not include any part of the final answer in these sections.
The actual response to the query must be entirely contained within the <output> tags.

Hint: ***Financial Reporting:**
```mermaid
graph TD
A[Articulating Purpose and Context] --> B[Collecting Input Data]
    B --> C[Processing Data]
    C --> D[Analyzing and Interpreting Processed Data]
    D --> E[Developing and Communicating Conclusions]
    E --> F[Doing Follow-Up]

    A --> |Defines goals, tools, and audience| B
    B --> |Gather data on economy and industry| C
    C --> |Use tools like ratios and charts| D
    D --> |Interpret data for conclusions| E
    F --> |Periodic review and iteration| A
```
***Fixed Income:***
```mermaid
graph TD
    A[Purpose and Scope] --> B3[Analyze Macro Conditions]
    B --> C[Assess Bond Features]
    C --> D[Risk and Yield Analysis]
    D --> E[Develop Recommendations]
    E --> F[Review Performance]

    %% Notes and detailed steps
    A --> |Set objectives| B
    B --> |Review interest rates and inflation| C
    C --> |Focus on duration, spread| D
    D --> |Assess scenarios| E
```
***Equity Investing:***
```mermaid
graph TD
    A[Objective Setting] --> B[Market and Sector Insights]
    B --> C[Industry Competitive Analysis]
    C --> D[Company Review]
    D --> E[Valuation and Risks]
    E --> F[Investment Decision]

    %% Step-specific highlights
    B --> |Look at growth patterns| C
    C --> |Evaluate competitors' positions| D
    D --> |Check financial health| E
    E --> |Combine insights into strategy| F
```
***Derivatives:***
```mermaid
graph TD
    A[Define Objective and Context] --> B[Identify Derivative Instrument]
    B --> C[Understand Contract Specifications]
    C --> D[Gather Market Data]
    D --> E[Apply Valuation Models]
    E --> F[Assess Risks: Market, Counterparty, etc.]
    F --> G[Construct Payoff Diagrams or Strategies]
    G --> H[Interpret Results and Make Recommendations]
    H --> I[Review, Monitor, and Adjust Strategies]

    %% Example labels or notes (optional)
    A --> |Hedging, speculation, arbitrage| B
    C --> |Features like notional amount, expiration| D
    D --> |Market prices, volatility, risk-free rates| E
    F --> |Sensitivity to Greeks: Delta, Gamma, Vega, etc.| G
    H --> |Adjust based on changing market conditions| I
```
***Economics:***
```mermaid
graph TD;
    A[Step 1: Question Breakdown] -->|Extract key terms| A1{Identify Topic}
    A1 -->|Micro: Supply & Demand, Market Structures| A2
    A1 -->|Macro: GDP, Growth, Policy, Trade| A3
    A1 -->|Currency & Regulation| A4

    A2 --> B1[Identify model: Elasticity, Cost Curves, Shutdown Points]
    A3 --> B2[Map to AD-AS, Business Cycles, Growth Theories]
    A4 --> B3[Assess Exchange Rates, Trade, Capital Flows, Regulation]

    B1 -->|Check for formula or concept?| C{Numerical or Conceptual}
    B2 --> C
    B3 --> C

    C -->|Numerical| D1[Extract data, apply formulas, check assumptions]
    C -->|Conceptual| D2[Analyze cause-effect, policy impact]

    D1 --> E[Step 4: Solution Development]
    D2 --> E
    E -->|Construct structured response| E1(Core insight + economic rationale)
    E -->|Consider alternative scenarios| E2(Assess different possibilities)

    E1 --> F[Step 5: Answer Validation]
    E2 --> F
    F -->|Check logic, principles, and assumptions| F1(Verify consistency)
    F1 -->|Ensure completeness & clarity| F2(Confirm answer structure)
```
***Quantitative Methods:***
```mermaid
graph TD
    A["Articulating Purpose and Context"] --> B["Collecting Input Data"]
    B --> C["Processing and Cleaning Data"]
    C --> D["Selecting Quantitative Models and Tools"]
    D --> E["Estimating Parameters and Testing Hypotheses"]
    E --> F["Interpreting Results and Communicating Findings"]
    F --> G["Monitoring and Model Reassessment"]
```
***Portfolio Management:***
```mermaid
graph TD
    A["Define Investment Objectives"] --> B["Establish Investment Constraints"]
    B --> C["Develop Strategic Asset Allocation"]
    C --> D["Incorporate Tactical Adjustments"]
    D --> E["Select and Optimize Securities"]
    E --> F["Execute Implementation and Trading"]
    F --> G["Measure Performance and Attribution"]
    G --> H["Monitor Risk and Compliance"]
    H --> I["Rebalance and Adjust Portfolio"]
```
***Alternative Investments:***
```mermaid
graph TD
    A["Define Investment Objectives and Mandate"] --> B["Identify Alternative Asset Classes"]
    B --> C["Conduct Manager and Strategy Due Diligence"]
    C --> D["Perform Valuation and Pricing Analysis"]
    D --> E["Assess Risk and Liquidity"]
    E --> F["Allocate Alternatives in Portfolio"]
    F --> G["Monitor Performance and Rebalance"]
```
***Corporate Issuer Analysis:***
```mermaid
graph TD
    A["Corporate Issuer Overview"] --> B["Industry Classification"]
    B --> C["Sector Trends and Competitive Landscape"]
    A --> D["Financial Statement Analysis"]
    D --> E["Profitability, Liquidity, Leverage"]
    A --> F["Credit Risk Assessment"]
    F --> G["Rating Agencies and Default Probabilities"]
    A --> H["Capital Structure and Issuance History"]
    H --> I["Bond Issuances and Debt Maturities"]
    A --> J["Corporate Governance and Management"]
    J --> K["Board Quality and Managerial Competence"]
    A --> L["Valuation and Investment Analysis"]
    L --> M["DCF, Relative Valuation, Multiples"]
```
### Response Format:
<thinking>
[Think step by step and respond with your thinking and the correct answer (A , B , C , D & Fall or Rise), considering the specific sector.]
</thinking>

<output>
"sector": [The sector being addressed],
"question": [The financial question],
"answer": [Reflect and verify the final answer (A , B , C , D & Fall or Rise )]
</output>"""

In [2]:
import pandas as pd

df = pd.read_csv("test.csv")
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [3]:
import re

def clean_query_column(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Removes predefined instructional patterns from the specified column in the DataFrame.

    Parameters:
    - df: pandas DataFrame containing the queries.
    - column_name: the name of the column to clean.

    Returns:
    - Cleaned pandas DataFrame.
    """
    # Define regex patterns to remove
    patterns = [
        r"Please respond with the exact answer A, B, C or D only\. Do not be verbose or provide extra information\.",
        r"โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม",
        r"อย่าอธิบายเยิ่นเย้อหรือให้ข้อมูลเพิ่มเติม"
    ]

    # Combine patterns into a single regex
    combined_pattern = "|".join(patterns)

    # Apply regex substitution to the specified column
    df[column_name] = df[column_name].str.replace(combined_pattern, "", regex=True).str.strip()

    return df

In [4]:
df = clean_query_column(df, "query")
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [5]:
df.loc[0].query

"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \n\nคำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้น\n\nตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing\n\nคำตอบ:"

In [6]:
def extract_choice(response_text: str):
    """Extracts the final answer choice from the model's response text, supporting EN/TH and A-D/Fall/Rise."""

    # Clean output formatting
    cleaned_text = re.sub(r"outputs:.*?\|.*?\|", "", response_text, flags=re.IGNORECASE)
    cleaned_text = cleaned_text.strip().replace("\n", " ")

    # Patterns that match common answer signals
    patterns = [
        r"(?:Final answer:|Answer:|คำตอบ:)\s*(A|B|C|D|Fall|Rise|ขึ้น|ลง)\b"
    ]

    for pattern in patterns:
        match = re.search(pattern, cleaned_text, flags=re.IGNORECASE)
        if match:
            choice = match.group(1)
            return translate_choice(choice)

    # Fallback: last standalone mention of a valid choice
    fallback = re.findall(r"\b(A|B|C|D|Fall|Rise|ขึ้น|ลง)\b", cleaned_text, flags=re.IGNORECASE)
    if fallback:
        return translate_choice(fallback[-1])

    return None

def translate_choice(choice: str) -> str:
    """Translates Thai terms to English and normalizes answer casing."""
    choice = choice.strip().lower()

    mapping = {
        "a": "A", "b": "B", "c": "C", "d": "D",
        "fall": "Fall", "rise": "Rise",
        "ลง": "Fall", "ขึ้น": "Rise"
    }

    return mapping.get(choice, None)

In [7]:
import torch
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "GGital/PeeJa", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 8192,
    load_in_4bit= False,
    load_in_8bit = True,
    dtype = torch.bfloat16
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
messages = [
    {"role" : "system" , "content" : PROMPTS_SYS},
    {"role" : "user", "content" : df.loc[7].query}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

output = model.generate(
    **inputs,
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
)

generated = output[0][inputs['input_ids'].shape[-1]:]
response = tokenizer.decode(generated, skip_special_tokens=True)
print(response)

**Question:** Analyze the data and tweets to summarize whether the closing price of $GS will rise or fall on 2017-12-20. Confirm whether it will rise or fall.

**Step-by-step Analysis:**

1. **Examine the Historical Data for $GS (2017-12-06 to 2017-12-19):**
   - The data provides daily open, high, low, close, adjusted close, and percentage changes for $GS.
   - The closing prices on the given dates are:
     - 2017-12-06: -1.0
     - 2017-12-07: 1.1
     - 2017-12-08: 0.7
     - 2017-12-11: -0.1
     - 2017-12-12: 3.0
     - 2017-12-13: -0.8
     - 2017-12-14: -0.0
     - 2017-12-15: 0.7
     - 2017-12-18: 1.1
     - 2017-12-19: -1.4
   - The closing price on 2017-12-19 is **-1.4**, which is lower than the previous day (2017-12-18: 1.1).

2. **Analyze the Trend:**
   - The closing prices show a mixed trend with fluctuations over the period.
   - There is a **drop on 2017-12-19** (-1.4), which suggests a **bearish signal**.
   - However, the previous day (2017-12-18) showed a **positiv

In [10]:
from tqdm import tqdm

def run_model_on_dataframe(df: pd.DataFrame, query_column: str = "query") -> pd.DataFrame:
    responses = []
    choices = []

    for i in tqdm(range(len(df)), desc="Processing queries"):
        query = df.loc[i, query_column]

        messages = [
            {"role": "system", "content": PROMPTS_SYS},
            {"role": "user", "content": query}
        ]

        try:
            # Apply chat template
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                enable_thinking=False  # For non-thinking generation
            )

            # Tokenize
            inputs = tokenizer(text, return_tensors="pt").to("cuda")

            # Generate output
            output = model.generate(
                **inputs,
                max_new_tokens=8192,
                temperature=0.7,
                top_p=0.8,
                top_k=20
            )

            # Decode only generated tokens (excluding prompt)
            generated = output[0][inputs['input_ids'].shape[-1]:]
            response = tokenizer.decode(generated, skip_special_tokens=True)

            responses.append(response)
            choices.append(extract_choice(response))

        except Exception as e:
            responses.append(str(e))
            choices.append(None)

    df["response"] = responses
    df["predicted_choice"] = choices
    return df


In [ ]:
result_df = run_model_on_dataframe(df)
result_df

Processing queries:   5%|▌         | 25/499 [25:45<9:50:15, 74.72s/it]

# Fincot

In [ ]:
PROMPTS_SYS = """You are a financial analyst taking a test to evaluate your knowledge of finance of different topics in finance. You think step by step approach with reflection to answer queries.

Follow these steps:
1. Think through the problem step by step reflect and verify while reasoning within the <thinking> tags.
2. Please and put the answer your final, concise answer within the <output> tags.

The <thinking> sections are for your internal reasoning process only.
Do not include any part of the final answer in these sections.
The actual response to the query must be entirely contained within the <output> tags.

Hint: ***Financial Reporting:**
```mermaid
graph TD
A[Articulating Purpose and Context] --> B[Collecting Input Data]
    B --> C[Processing Data]
    C --> D[Analyzing and Interpreting Processed Data]
    D --> E[Developing and Communicating Conclusions]
    E --> F[Doing Follow-Up]

    A --> |Defines goals, tools, and audience| B
    B --> |Gather data on economy and industry| C
    C --> |Use tools like ratios and charts| D
    D --> |Interpret data for conclusions| E
    F --> |Periodic review and iteration| A
```
***Fixed Income:***
```mermaid
graph TD
    A[Purpose and Scope] --> B3[Analyze Macro Conditions]
    B --> C[Assess Bond Features]
    C --> D[Risk and Yield Analysis]
    D --> E[Develop Recommendations]
    E --> F[Review Performance]

    %% Notes and detailed steps
    A --> |Set objectives| B
    B --> |Review interest rates and inflation| C
    C --> |Focus on duration, spread| D
    D --> |Assess scenarios| E
```
***Equity Investing:***
```mermaid
graph TD
    A[Objective Setting] --> B[Market and Sector Insights]
    B --> C[Industry Competitive Analysis]
    C --> D[Company Review]
    D --> E[Valuation and Risks]
    E --> F[Investment Decision]

    %% Step-specific highlights
    B --> |Look at growth patterns| C
    C --> |Evaluate competitors' positions| D
    D --> |Check financial health| E
    E --> |Combine insights into strategy| F
```
***Derivatives:***
```mermaid
graph TD
    A[Define Objective and Context] --> B[Identify Derivative Instrument]
    B --> C[Understand Contract Specifications]
    C --> D[Gather Market Data]
    D --> E[Apply Valuation Models]
    E --> F[Assess Risks: Market, Counterparty, etc.]
    F --> G[Construct Payoff Diagrams or Strategies]
    G --> H[Interpret Results and Make Recommendations]
    H --> I[Review, Monitor, and Adjust Strategies]

    %% Example labels or notes (optional)
    A --> |Hedging, speculation, arbitrage| B
    C --> |Features like notional amount, expiration| D
    D --> |Market prices, volatility, risk-free rates| E
    F --> |Sensitivity to Greeks: Delta, Gamma, Vega, etc.| G
    H --> |Adjust based on changing market conditions| I
```
***Economics:***
```mermaid
graph TD;
    A[Step 1: Question Breakdown] -->|Extract key terms| A1{Identify Topic}
    A1 -->|Micro: Supply & Demand, Market Structures| A2
    A1 -->|Macro: GDP, Growth, Policy, Trade| A3
    A1 -->|Currency & Regulation| A4

    A2 --> B1[Identify model: Elasticity, Cost Curves, Shutdown Points]
    A3 --> B2[Map to AD-AS, Business Cycles, Growth Theories]
    A4 --> B3[Assess Exchange Rates, Trade, Capital Flows, Regulation]

    B1 -->|Check for formula or concept?| C{Numerical or Conceptual}
    B2 --> C
    B3 --> C

    C -->|Numerical| D1[Extract data, apply formulas, check assumptions]
    C -->|Conceptual| D2[Analyze cause-effect, policy impact]

    D1 --> E[Step 4: Solution Development]
    D2 --> E
    E -->|Construct structured response| E1(Core insight + economic rationale)
    E -->|Consider alternative scenarios| E2(Assess different possibilities)

    E1 --> F[Step 5: Answer Validation]
    E2 --> F
    F -->|Check logic, principles, and assumptions| F1(Verify consistency)
    F1 -->|Ensure completeness & clarity| F2(Confirm answer structure)
```
***Quantitative Methods:***
```mermaid
graph TD
    A["Articulating Purpose and Context"] --> B["Collecting Input Data"]
    B --> C["Processing and Cleaning Data"]
    C --> D["Selecting Quantitative Models and Tools"]
    D --> E["Estimating Parameters and Testing Hypotheses"]
    E --> F["Interpreting Results and Communicating Findings"]
    F --> G["Monitoring and Model Reassessment"]
```
***Portfolio Management:***
```mermaid
graph TD
    A["Define Investment Objectives"] --> B["Establish Investment Constraints"]
    B --> C["Develop Strategic Asset Allocation"]
    C --> D["Incorporate Tactical Adjustments"]
    D --> E["Select and Optimize Securities"]
    E --> F["Execute Implementation and Trading"]
    F --> G["Measure Performance and Attribution"]
    G --> H["Monitor Risk and Compliance"]
    H --> I["Rebalance and Adjust Portfolio"]
```
***Alternative Investments:***
```mermaid
graph TD
    A["Define Investment Objectives and Mandate"] --> B["Identify Alternative Asset Classes"]
    B --> C["Conduct Manager and Strategy Due Diligence"]
    C --> D["Perform Valuation and Pricing Analysis"]
    D --> E["Assess Risk and Liquidity"]
    E --> F["Allocate Alternatives in Portfolio"]
    F --> G["Monitor Performance and Rebalance"]
```
***Corporate Issuer Analysis:***
```mermaid
graph TD
    A["Corporate Issuer Overview"] --> B["Industry Classification"]
    B --> C["Sector Trends and Competitive Landscape"]
    A --> D["Financial Statement Analysis"]
    D --> E["Profitability, Liquidity, Leverage"]
    A --> F["Credit Risk Assessment"]
    F --> G["Rating Agencies and Default Probabilities"]
    A --> H["Capital Structure and Issuance History"]
    H --> I["Bond Issuances and Debt Maturities"]
    A --> J["Corporate Governance and Management"]
    J --> K["Board Quality and Managerial Competence"]
    A --> L["Valuation and Investment Analysis"]
    L --> M["DCF, Relative Valuation, Multiples"]
```
### Response Format:
<thinking>
[Think step by step and respond with your thinking and the correct answer (A , B , C , D & Fall or Rise), considering the specific sector.]
</thinking>

<output>
"sector": [The sector being addressed],
"question": [The financial question],
"answer": [Reflect and verify the final answer (A , B , C , D & Fall or Rise )]
</output>"""

In [ ]:
PROMPTS_SYS = """ You are a CFA (chartered financial analyst) taking a test to
 evaluate your knowledge of finance.
 You will be given a question along
 with three possible answers (A, B, C, D, Fall, Rise).
 Before answering, you should think
 through the question step-by-step.
 Explain your reasoning at each step
 towards answering the question. If
 calculation is required, do each
 step of the calculation as a step
 in your reasoning.
 Indicate the correct answer (A, B, C, D, Fall, Rise).
 <output>
 "answer": [Reflect and verify the final answer (A , B , C , D & Fall or Rise )]
 </output>
"""

In [ ]:
import pandas as pd

df = pd.read_csv("test.csv")
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [ ]:
import re

def clean_query_column(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Removes predefined instructional patterns from the specified column in the DataFrame.

    Parameters:
    - df: pandas DataFrame containing the queries.
    - column_name: the name of the column to clean.

    Returns:
    - Cleaned pandas DataFrame.
    """
    # Define regex patterns to remove
    patterns = [
        r"Please respond with the exact answer A, B, C or D only\. Do not be verbose or provide extra information\.",
        r"โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม",
        r"อย่าอธิบายเยิ่นเย้อหรือให้ข้อมูลเพิ่มเติม"
    ]

    # Combine patterns into a single regex
    combined_pattern = "|".join(patterns)

    # Apply regex substitution to the specified column
    df[column_name] = df[column_name].str.replace(combined_pattern, "", regex=True).str.strip()

    return df

In [ ]:
df = clean_query_column(df, "query")
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [ ]:
from openai import OpenAI
from google.colab import userdata
# Initialize the client with your API key and the OpenTyphoon base URL
client = OpenAI(
    api_key=userdata.get('TYPHOON_API_KEY'),
    base_url="https://api.opentyphoon.ai/v1"
)

# Make a completion request
response = client.chat.completions.create(
    model="typhoon-v2-70b-instruct",
    messages=[
        {"role": "system", "content": PROMPTS_SYS},
        {"role": "user", "content": df.loc[6].query}
    ],
    max_tokens=1024,
    temperature=0.0
)

# Print the response
print(response.choices[0].message.content)

<thinking>
To answer this question, we need to understand the concept of joint creation of value and how it relates to customer involvement in the marketing process. The term "co-creation" is often used in marketing to describe situations where customers actively participate in creating their own experiences or products. This concept emphasizes the importance of collaboration between companies and their customers to generate value that is mutually beneficial. 

Let's break down the options provided:

A: Interactive marketing refers to the use of digital channels to engage customers in a two-way communication. While interactive marketing can involve customer participation, it does not specifically focus on the joint creation of value.

B: Co-creation experience directly refers to the process of jointly creating value with customers. This option aligns closely with the definition of co-creation, which involves customers in the design, production, and delivery of personalized experiences.

In [ ]:
df.loc[3].query

'ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้องแม่นยำคือ A, B, C หรือ D เท่านั้น \nคำถาม: หาก GDP ที่แท้จริงปัจจุบันคือ 5000 ดอลลาร์ และ GDP ที่แท้จริงของการจ้างงานเต็มอัตราอยู่ที่ 4000 ดอลลาร์ นโยบายชุดใดต่อไปนี้ที่มีแนวโน้มมากที่สุดที่จะนำพาเศรษฐกิจมาถึงจุดนี้\nตัวเลือกคำตอบ: A: การลดภาษีและอัตราคิดลดที่ต่ำลง, B: การเพิ่มการใช้จ่ายของรัฐบาลและการเพิ่มภาษี, C: การลดภาษีและการขายพันธบัตรในการดำเนินงานในตลาดเปิด, D: การเพิ่มการใช้จ่ายของรัฐบาลและการเพิ่มอัตราคิดลด\nคำตอบ:'

In [ ]:
from openai import OpenAI
from google.colab import userdata
# Initialize the client with your API key and the OpenTyphoon base URL
client = OpenAI(
    api_key=userdata.get('OPENROUTER'),
    base_url="https://openrouter.ai/api/v1"
)

# Make a completion request
response = client.chat.completions.create(
    model="qwen/qwen3-8b",
    messages=[
        {"role": "system", "content": PROMPTS_SYS},
        {"role": "user", "content": df.loc[3].query}
    ],
    max_tokens=32678,
    temperature=0.7,  # suggest for non-thinking mode: https://huggingface.co/Qwen/Qwen3-14B
    top_p=0.8,
    extra_body={
        "top_k": 20,
        "min_p": 0.0,
        "chat_template_kwargs": {
            "enable_thinking": True       # hard-switch OFF the CoT stream
        }
    }

)

# Print the response
print(response.choices[0].message.content)

"answer": "A"


In [ ]:
def extract_choice(response_text: str):
    """Extracts the final answer choice from the model's response text, supporting EN/TH and A-D/Fall/Rise."""

    # Clean output formatting
    cleaned_text = re.sub(r"outputs:.*?\|.*?\|", "", response_text, flags=re.IGNORECASE)
    cleaned_text = cleaned_text.strip().replace("\n", " ")

    # Patterns that match common answer signals
    patterns = [
        r"(?:Final answer:|Answer:|คำตอบ:)\s*(A|B|C|D|Fall|Rise|ขึ้น|ลง)\b"
    ]

    for pattern in patterns:
        match = re.search(pattern, cleaned_text, flags=re.IGNORECASE)
        if match:
            choice = match.group(1)
            return translate_choice(choice)

    # Fallback: last standalone mention of a valid choice
    fallback = re.findall(r"\b(A|B|C|D|Fall|Rise|ขึ้น|ลง)\b", cleaned_text, flags=re.IGNORECASE)
    if fallback:
        return translate_choice(fallback[-1])

    return None

def translate_choice(choice: str) -> str:
    """Translates Thai terms to English and normalizes answer casing."""
    choice = choice.strip().lower()

    mapping = {
        "a": "A", "b": "B", "c": "C", "d": "D",
        "fall": "Fall", "rise": "Rise",
        "ลง": "Fall", "ขึ้น": "Rise"
    }

    return mapping.get(choice, None)

In [ ]:
from tqdm import tqdm
from openai import OpenAI
from google.colab import userdata

def run_model_on_dataframe(df: pd.DataFrame, query_column: str = "query") -> pd.DataFrame:
    responses = []
    choices = []

    for i in tqdm(range(len(df)), desc="Processing queries"):
        query = df.loc[i, query_column]

        try:
            response = client.chat.completions.create(
                model="qwen/qwen3-32b",
                messages=[
                    {"role": "system", "content": PROMPTS_SYS},
                    {"role": "user", "content": query}
                ],
                max_tokens=32678,
                temperature=0.3,  # suggest for non-thinking mode: https://huggingface.co/Qwen/Qwen3-14B
                top_p=0.8,
                extra_body={
                    "top_k": 20,
                    "min_p": 0.0,
                    "chat_template_kwargs": {
                        "enable_thinking": True       # hard-switch OFF the CoT stream
                    }
                }
            )

            content = response.choices[0].message.content
            responses.append(content)
            choices.append(extract_choice(content))

        except Exception as e:
            responses.append(str(e))
            choices.append(None)

    df["response"] = responses
    df["predicted_choice"] = choices
    return df

In [ ]:
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [ ]:
API_URL = "https://openrouter.ai/api/v1/chat/completions"
API_KEY = userdata.get('OPENROUTER')

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:
import asyncio
import httpx
import pandas as pd
from tqdm.asyncio import tqdm_asyncio

BATCH_SIZE = 100

async def call_openai(session, query: str, row_id: int):
    payload = {
        "model": "deepseek/deepseek-r1-0528",
        "messages": [
            {"role": "system", "content": PROMPTS_SYS},
            {"role": "user", "content": query}
        ],
        "max_tokens": 32678,
        "temperature": 0.6,
        "top_p": 0.95,
        "extra_body": {
            "top_k": 20,
            "min_p": 0.0,
            "chat_template_kwargs": {
                "enable_thinking": True
            }
        }
    }

    try:
        response = await session.post(API_URL, json=payload, headers=HEADERS, timeout=60)
        response.raise_for_status()

        json_data = response.json()
        content = json_data["choices"][0]["message"]["content"]
        return row_id, content, extract_choice(content)

    except Exception as e:
        print(row_id)
        return row_id, f"[EXCEPTION] {str(e)}", None


async def run_model_on_dataframe_async(df: pd.DataFrame, query_column: str = "query") -> pd.DataFrame:
    responses = [None] * len(df)
    choices = [None] * len(df)

    async with httpx.AsyncClient() as session:
        for start in tqdm_asyncio(range(0, len(df), BATCH_SIZE), desc="Processing batches"):
            end = min(start + BATCH_SIZE, len(df))
            tasks = [
                call_openai(session, df.loc[i, query_column], i)
                for i in range(start, end)
            ]

            results = await asyncio.gather(*tasks)

            for row_id, response, choice in results:
                responses[row_id] = response
                choices[row_id] = choice

    df["response"] = responses
    df["predicted_choice"] = choices
    return df


## Testing Part

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

test_df = df.head(30)
test_df = await run_model_on_dataframe_async(test_df)
test_df

Processing queries: 100%|██████████| 30/30 [07:15<00:00, 14.52s/it]
/tmp/ipython-input-46-3209240464.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["response"] = responses
/tmp/ipython-input-46-3209240464.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["predicted_choice"] = choices


,id,query,response,predicted_choice
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \...","""answer"": ""D: Rehumanizing"" \n\n**Reasoning:**...",D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...,"""answer"": ""B""",B
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...,"""answer"": ""A""",A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",คำตอบ: A\n\n**เหตุผล:** \n1. **การวิเคราะห์สถ...,A
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w...",The question asks whether the closing price of...,Rise
5,ff5b5d2e-5fa1-4709-a9a7-681e4d4585bd,Answer the question with the appropriate optio...,The managerial functions are typically categor...,C
6,d7a45917-d0f9-476e-912d-ebc5af9333a1,Answer the question with the appropriate optio...,"""answer"": ""B: Co-creation experience.""",B
7,e625dbc8-f448-4c53-9a78-6c3f351b49c3,วิเคราะห์ข้อมูลและทวีตเพื่อสรุปว่าราคาปิดของ $...,"""answer"": [""Rise""]",Rise
8,9bea42e5-3c21-46dc-93f7-0017f382f7cf,ประเมินข้อมูลและทวีตเพื่อประเมินว่าราคาปิดของ ...,"""answer"": ""Rise"" \n**Reasoning**: \n1. **Dat...",Rise
9,0925a4d7-8546-46a8-834f-20f58f16bc99,"Read the questions and answers carefully, and ...",The scenario involves a charitable foundation ...,C


In [ ]:
test_df.loc[0].response

'"answer": "D: Rehumanizing" \n\n**Reasoning:**  \n1. **Understanding the Concepts:** The question refers to two job design strategies: *job enrichment* (increasing job depth by giving employees more responsibility and autonomy) and *job enlargement* (increasing job breadth by adding more varied tasks). Both aim to enhance employee capabilities and job satisfaction.  \n2. **Analyzing the Options:**  \n   - **A: Re-invigorating** focuses on restoring energy or motivation but does not directly relate to expanding job roles or responsibilities.  \n   - **B: Re-flourishing** is not a standard term in management theory and lacks clear relevance to job design.  \n   - **C: Revitalizing** implies renewing or energizing, which could align with motivation but does not explicitly address job structure or employee capacity building.  \n   - **D: Rehumanizing** refers to making work more human-centered, emphasizing autonomy, meaningful tasks, and personal growth—key aspects of both job enrichment 

In [ ]:
test_df.to_csv('output_test.csv' , index=False)

## Real run

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()  # Allow nested event loops
df = await run_model_on_dataframe_async(df)

Processing batches: 100%|██████████| 5/5 [26:22<00:00, 316.56s/it]


In [ ]:
df.to_csv('output_DeepSeek_R1_Batched.csv' , index=False)

In [ ]:
df

,id,query,response,predicted_choice
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \...",คำถามนี้เกี่ยวกับแนวคิดในการเสริมสร้างศักยภาพพ...,C
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...,### Step-by-Step Reasoning:\n\n1. **Understand...,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...,### Step-by-Step Reasoning:\n\nTo answer this ...,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",### ขั้นตอนการคิด\nคำถาม: หาก GDP ที่แท้จริงปั...,None
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w...",### Step-by-Step Reasoning\n\nTo project wheth...,Fall
...,...,...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...,### Step-by-Step Reasoning\n\nThe question ask...,D
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D...",D\n\n*หมายเหตุ: คำตอบที่ถูกต้องคือ **D: Contin...,D
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",D,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...,Based on a thorough analysis of the historical...,Fall


In [ ]:
import pandas as pd

def filter_none_rows(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Filters and returns rows from the DataFrame where the specified column is None/NaN.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to check for None/NaN.

    Returns:
        pd.DataFrame: A DataFrame containing only rows where the specified column is None/NaN.
    """
    return df[df[column_name].isna()]


In [ ]:
nan_df = filter_none_rows(submission_extracted , "answer")

In [ ]:
nan_df

,id,answer
157,55195adb-c94f-433b-9525-009a86a4b325,NaN
218,9585af54-a3c1-4ace-b532-0f200fb804f0,NaN
277,4fc850d7-7c29-4ca3-9614-426e62364660,NaN
355,cde3aba2-c5a4-47ec-835c-92f9b062c4af,NaN


## Replace with another Model

In [ ]:
import pandas as pd

def overwrite_column_with_specified_data(source_df: pd.DataFrame,
                                         target_df: pd.DataFrame,
                                         match_column: str,
                                         value_column: str,
                                         target_column: str) -> pd.DataFrame:
    """
    Overwrite values in target_df[target_column] using source_df[value_column]
    if current target value is 'Fall', 'Rise', or None/empty. Leave 'A'-'D' untouched.

    Args:
        source_df (pd.DataFrame): DataFrame with new values.
        target_df (pd.DataFrame): DataFrame to update.
        match_column (str): Column to match on.
        value_column (str): Column from source_df to get new value.
        target_column (str): Column in target_df to update.

    Returns:
        pd.DataFrame: Updated DataFrame.
    """
    result_df = target_df.copy()
    source_map = source_df.set_index(match_column)[value_column]

    for idx, row in result_df.iterrows():
        key = row[match_column]
        current_value = row[target_column]

        # Only replace if current value is Fall, Rise, or empty/None
        if current_value in ['Fall', 'Rise', '', None] or pd.isna(current_value):
            if key in source_map:
                new_value = source_map[key]
                if pd.notna(new_value) and str(new_value).strip() != "":
                    result_df.at[idx, target_column] = new_value

    return result_df

In [ ]:
ground = pd.read_csv("KBANK.csv")
ground

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,A
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,D
495,73c720b5-1101-4790-af52-3366823e1d32,B
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Rise


In [ ]:
merged_df = overwrite_column_with_specified_data(ground ,
                                                 df,
                                                 "id",
                                                 "answer",
                                                 "predicted_choice")
merged_df

,id,query,response,predicted_choice
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \...","""answer"": ""D: Rehumanizing"" \n\n**เหตุผล:** \...",D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...,"""answer"": ""B""",B
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...,"""answer"": ""A""",A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...","""answer"": [""C""]",C
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w...",,Rise
...,...,...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...,The general-to-specific (LSE/Hendry) approach ...,D
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D...",,B
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",D: สถานที่หรือกิจกรรมการจัดจำหน่าย,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...,,Rise


In [ ]:
nan_df = filter_none_rows(merged_df , "predicted_choice")
nan_df

,id,query,response,predicted_choice


In [ ]:
merged_df.to_csv("output_merged.csv" , index=False)

## Running on missing

In [ ]:
def run_model_on_dataframe_missing_only(df: pd.DataFrame, query_column: str = "query") -> pd.DataFrame:
    """
    Runs the model only on rows where predicted_choice is None.
    Updates the 'response' and 'predicted_choice' columns in-place.
    """
    for i in tqdm(df[df["predicted_choice"].isna()].index, desc="Processing missing predictions"):
        query = df.loc[i, query_column]

        try:
            response = client.chat.completions.create(
                model="typhoon-v2-70b-instruct",
                messages=[
                    {"role": "system", "content": PROMPTS_SYS},
                    {"role": "user", "content": query}
                ],
                max_tokens=700,
                temperature=0.0
            )

            content = response.choices[0].message.content
            df.at[i, "response"] = content
            df.at[i, "predicted_choice"] = extract_choice(content)

        except Exception as e:
            df.at[i, "response"] = str(e)
            df.at[i, "predicted_choice"] = None

    return df


In [ ]:
df = run_model_on_dataframe_missing_only(df)

Processing missing predictions: 100%|██████████| 81/81 [14:47<00:00, 10.96s/it]


In [ ]:
nan_df = filter_none_rows(df , "predicted_choice")
nan_df

,id,query,response,predicted_choice
53,d9825631-a11e-463b-988f-4e070213f92b,วิเคราะห์ข้อมูลและทวีตต่างๆ เพื่อพิจารณาว่าราค...,<thinking>\nเราต้องวิเคราะห์ข้อมูลและทวีตต่างๆ...,None
234,3c302b63-0d15-43ff-b731-e94b70574213,จากข้อมูลและทวีตต่างๆ เราสามารถคาดการณ์ได้หรือ...,<thinking>\nเราจะใช้ข้อมูลที่ให้มาเพื่อวิเคราะ...,None
277,4fc850d7-7c29-4ca3-9614-426e62364660,ประเมินข้อมูลและทวีตเพื่อประมาณว่าราคาปิดของ $...,<thinking>\nเราต้องประเมินข้อมูลและทวีตเพื่อปร...,None
355,cde3aba2-c5a4-47ec-835c-92f9b062c4af,ประเมินข้อมูลและทวีตเพื่อประมาณว่าราคาปิดของ $...,<thinking>\nจากข้อมูลที่ให้มา เราสามารถเห็นได้...,None
438,794c0de8-809e-46b2-a39a-e2746504de92,จากข้อมูลและทวีตที่ให้มา คุณจะคาดการณ์ได้หรือไ...,<thinking>\nจากข้อมูลที่ให้มา เราจะต้องวิเคราะ...,None


#Qwen3 Fincot

In [ ]:
PROMPTS_SYS = """You are a financial analyst taking a test to evaluate your knowledge of finance of different topics in finance. You think step by step approach with reflection to answer queries.

Follow these steps:
1. Think through the problem step by step reflect and verify while reasoning within the <thinking> tags.
2. Please and put the answer your final, concise answer within the <output> tags.

The <thinking> sections are for your internal reasoning process only.
Do not include any part of the final answer in these sections.
The actual response to the query must be entirely contained within the <output> tags.

Hint: ***Financial Reporting:**
```mermaid
graph TD
A[Articulating Purpose and Context] --> B[Collecting Input Data]
    B --> C[Processing Data]
    C --> D[Analyzing and Interpreting Processed Data]
    D --> E[Developing and Communicating Conclusions]
    E --> F[Doing Follow-Up]

    A --> |Defines goals, tools, and audience| B
    B --> |Gather data on economy and industry| C
    C --> |Use tools like ratios and charts| D
    D --> |Interpret data for conclusions| E
    F --> |Periodic review and iteration| A
```
***Fixed Income:***
```mermaid
graph TD
    A[Purpose and Scope] --> B3[Analyze Macro Conditions]
    B --> C[Assess Bond Features]
    C --> D[Risk and Yield Analysis]
    D --> E[Develop Recommendations]
    E --> F[Review Performance]

    %% Notes and detailed steps
    A --> |Set objectives| B
    B --> |Review interest rates and inflation| C
    C --> |Focus on duration, spread| D
    D --> |Assess scenarios| E
```
***Equity Investing:***
```mermaid
graph TD
    A[Objective Setting] --> B[Market and Sector Insights]
    B --> C[Industry Competitive Analysis]
    C --> D[Company Review]
    D --> E[Valuation and Risks]
    E --> F[Investment Decision]

    %% Step-specific highlights
    B --> |Look at growth patterns| C
    C --> |Evaluate competitors' positions| D
    D --> |Check financial health| E
    E --> |Combine insights into strategy| F
```
***Derivatives:***
```mermaid
graph TD
    A[Define Objective and Context] --> B[Identify Derivative Instrument]
    B --> C[Understand Contract Specifications]
    C --> D[Gather Market Data]
    D --> E[Apply Valuation Models]
    E --> F[Assess Risks: Market, Counterparty, etc.]
    F --> G[Construct Payoff Diagrams or Strategies]
    G --> H[Interpret Results and Make Recommendations]
    H --> I[Review, Monitor, and Adjust Strategies]

    %% Example labels or notes (optional)
    A --> |Hedging, speculation, arbitrage| B
    C --> |Features like notional amount, expiration| D
    D --> |Market prices, volatility, risk-free rates| E
    F --> |Sensitivity to Greeks: Delta, Gamma, Vega, etc.| G
    H --> |Adjust based on changing market conditions| I
```
***Economics:***
```mermaid
graph TD;
    A[Step 1: Question Breakdown] -->|Extract key terms| A1{Identify Topic}
    A1 -->|Micro: Supply & Demand, Market Structures| A2
    A1 -->|Macro: GDP, Growth, Policy, Trade| A3
    A1 -->|Currency & Regulation| A4

    A2 --> B1[Identify model: Elasticity, Cost Curves, Shutdown Points]
    A3 --> B2[Map to AD-AS, Business Cycles, Growth Theories]
    A4 --> B3[Assess Exchange Rates, Trade, Capital Flows, Regulation]

    B1 -->|Check for formula or concept?| C{Numerical or Conceptual}
    B2 --> C
    B3 --> C

    C -->|Numerical| D1[Extract data, apply formulas, check assumptions]
    C -->|Conceptual| D2[Analyze cause-effect, policy impact]

    D1 --> E[Step 4: Solution Development]
    D2 --> E
    E -->|Construct structured response| E1(Core insight + economic rationale)
    E -->|Consider alternative scenarios| E2(Assess different possibilities)

    E1 --> F[Step 5: Answer Validation]
    E2 --> F
    F -->|Check logic, principles, and assumptions| F1(Verify consistency)
    F1 -->|Ensure completeness & clarity| F2(Confirm answer structure)
```
***Quantitative Methods:***
```mermaid
graph TD
    A["Articulating Purpose and Context"] --> B["Collecting Input Data"]
    B --> C["Processing and Cleaning Data"]
    C --> D["Selecting Quantitative Models and Tools"]
    D --> E["Estimating Parameters and Testing Hypotheses"]
    E --> F["Interpreting Results and Communicating Findings"]
    F --> G["Monitoring and Model Reassessment"]
```
***Portfolio Management:***
```mermaid
graph TD
    A["Define Investment Objectives"] --> B["Establish Investment Constraints"]
    B --> C["Develop Strategic Asset Allocation"]
    C --> D["Incorporate Tactical Adjustments"]
    D --> E["Select and Optimize Securities"]
    E --> F["Execute Implementation and Trading"]
    F --> G["Measure Performance and Attribution"]
    G --> H["Monitor Risk and Compliance"]
    H --> I["Rebalance and Adjust Portfolio"]
```
***Alternative Investments:***
```mermaid
graph TD
    A["Define Investment Objectives and Mandate"] --> B["Identify Alternative Asset Classes"]
    B --> C["Conduct Manager and Strategy Due Diligence"]
    C --> D["Perform Valuation and Pricing Analysis"]
    D --> E["Assess Risk and Liquidity"]
    E --> F["Allocate Alternatives in Portfolio"]
    F --> G["Monitor Performance and Rebalance"]
```
***Corporate Issuer Analysis:***
```mermaid
graph TD
    A["Corporate Issuer Overview"] --> B["Industry Classification"]
    B --> C["Sector Trends and Competitive Landscape"]
    A --> D["Financial Statement Analysis"]
    D --> E["Profitability, Liquidity, Leverage"]
    A --> F["Credit Risk Assessment"]
    F --> G["Rating Agencies and Default Probabilities"]
    A --> H["Capital Structure and Issuance History"]
    H --> I["Bond Issuances and Debt Maturities"]
    A --> J["Corporate Governance and Management"]
    J --> K["Board Quality and Managerial Competence"]
    A --> L["Valuation and Investment Analysis"]
    L --> M["DCF, Relative Valuation, Multiples"]
```
### Response Format:
<thinking>
[Think step by step and respond with your thinking and the correct answer (A , B , C or D ), considering the specific sector.]
</thinking>

<output>
"sector": [The sector being addressed],
"question": [The financial question],
"answer": [Reflect and verify the final answer (A , B , C or D )]
</output>"""

In [ ]:
import pandas as pd

df = pd.read_csv("test.csv")
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [ ]:
import re

def clean_query_column(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Removes predefined instructional patterns from the specified column in the DataFrame.

    Parameters:
    - df: pandas DataFrame containing the queries.
    - column_name: the name of the column to clean.

    Returns:
    - Cleaned pandas DataFrame.
    """
    # Define regex patterns to remove
    patterns = [
        r"Please respond with the exact answer A, B, C or D only\. Do not be verbose or provide extra information\.",
        r"โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม"
    ]

    # Combine patterns into a single regex
    combined_pattern = "|".join(patterns)

    # Apply regex substitution to the specified column
    df[column_name] = df[column_name].str.replace(combined_pattern, "", regex=True).str.strip()

    return df

In [ ]:
df = clean_query_column(df, "query")
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_ID = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16,
        device_map="auto")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
def extract_choice(response_text: str):
    """Extracts the final answer choice from the model's response text, supporting EN/TH and A-D/Fall/Rise."""

    # Clean output formatting
    cleaned_text = re.sub(r"outputs:.*?\|.*?\|", "", response_text, flags=re.IGNORECASE)
    cleaned_text = cleaned_text.strip().replace("\n", " ")

    # Patterns that match common answer signals
    patterns = [
        r"(?:Final answer:|Answer:|คำตอบ:)\s*(A|B|C|D|Fall|Rise|ขึ้น|ลง)\b"
    ]

    for pattern in patterns:
        match = re.search(pattern, cleaned_text, flags=re.IGNORECASE)
        if match:
            choice = match.group(1)
            return translate_choice(choice)

    # Fallback: last standalone mention of a valid choice
    fallback = re.findall(r"\b(A|B|C|D|Fall|Rise|ขึ้น|ลง)\b", cleaned_text, flags=re.IGNORECASE)
    if fallback:
        return translate_choice(fallback[-1])

    return None

def translate_choice(choice: str) -> str:
    """Translates Thai terms to English and normalizes answer casing."""
    choice = choice.strip().lower()

    mapping = {
        "a": "A", "b": "B", "c": "C", "d": "D",
        "fall": "Fall", "rise": "Rise",
        "ลง": "Fall", "ขึ้น": "Rise"
    }

    return mapping.get(choice, None)

In [ ]:
from tqdm import tqdm

def run_qwen_on_dataframe(df: pd.DataFrame, query_column: str = "query") -> pd.DataFrame:
    """
    Runs Qwen3-8B on every row in the DataFrame.
    Fills 'response' and 'predicted_choice' columns.
    """
    responses = []
    choices = []

    for i in tqdm(range(len(df)), desc="Qwen3-8B Processing"):
        prompt = df.loc[i, query_column]
        messages = [{"role": "system", "content": PROMPTS_SYS},
         {"role": "user", "content": prompt}]

        try:
            # Apply chat template
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                enable_thinking=True
            )
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

            # Generate
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=8192,
                top_p=0.8,
                temperature=0.1,
                top_k=20,
            )
            output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

            # Split on </think> token ID (151668)
            try:
                index = len(output_ids) - output_ids[::-1].index(151668)
            except ValueError:
                index = 0

            thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
            content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

            responses.append(content)
            choices.append(extract_choice(content))

        except Exception as e:
            responses.append(str(e))
            choices.append(None)

    # Store to DataFrame
    df["response"] = responses
    df["predicted_choice"] = choices

    return df

In [ ]:
from tqdm import tqdm

def vllm_run_qwen_on_dataframe(df: pd.DataFrame, query_column: str = "query") -> pd.DataFrame:
    """
    Runs Qwen3-8B on every row in the DataFrame.
    Fills 'response' and 'predicted_choice' columns.
    """
    responses = []
    choices = []

    for i in tqdm(range(len(df)), desc="Qwen3-8B Processing"):
        prompt = df.loc[i, query_column]
        messages = [{"role": "system", "content": PROMPTS_SYS},
         {"role": "user", "content": prompt}]

        try:
            sampling_params = SamplingParams(temperature=0, top_p=0.8, max_tokens=16384)

            outputs = llm.chat(messages, sampling_params=sampling_params)

            content = outputs[0].outputs[0].text

            responses.append(content)
            choices.append(extract_choice(content))

        except Exception as e:
            responses.append(str(e))
            choices.append(None)

    # Store to DataFrame
    df["response"] = responses
    df["predicted_choice"] = choices

    return df

In [ ]:
df = run_qwen_on_dataframe(df)

Qwen3-8B Processing:  60%|██████    | 300/499 [7:49:51<5:03:27, 91.49s/it] 

In [ ]:
df.to_csv("Qwen_output.csv" , index=False)

In [ ]:
df.to_csv('./drive/MyDrive/Qwen_output.csv', index=False)

# Creating Submission

In [ ]:
submission = pd.read_csv("output_Qwen3_32B_Batched.csv")

In [ ]:
submission

,id,query,response,predicted_choice
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D \...","""answer"": ""C""",C
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...,"""answer"": ""B""",B
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...,The question asks for the most effective metho...,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",C,C
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w...",The historical data shows a recent upward tren...,Rise
...,...,...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...,"The LSE (Hendry) approach, which follows a gen...",B
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D...",A: Expectancy,A
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",D,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...,To determine whether the closing price of Bank...,Rise


In [ ]:
def extract_valid_answers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts 'id' and 'predicted_choice' (renamed to 'answer') columns.
    Prints the id of any row where the predicted_choice is invalid (not A, B, C, D, Rise, or Fall).

    Returns:
        A new DataFrame with columns: id, answer
    """
    valid_choices = {"A", "B", "C", "D", "Rise", "Fall"}

    # Rename and filter columns
    output_df = df[["id", "predicted_choice"]].copy()
    output_df.rename(columns={"predicted_choice": "answer"}, inplace=True)

    # Check for invalid answers
    for i, row in output_df.iterrows():
        if row["answer"] not in valid_choices:
            print(f"Invalid answer at id: {row['id']} -> {row['answer']}")

    return output_df


In [ ]:
submission_extracted = extract_valid_answers(submission)
submission_extracted

Invalid answer at id: 55195adb-c94f-433b-9525-009a86a4b325 -> nan
Invalid answer at id: 9585af54-a3c1-4ace-b532-0f200fb804f0 -> nan
Invalid answer at id: 4fc850d7-7c29-4ca3-9614-426e62364660 -> nan
Invalid answer at id: cde3aba2-c5a4-47ec-835c-92f9b062c4af -> nan


,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,C
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,C
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,B
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Rise


In [ ]:
submission_extracted.to_csv("submission_Qwen3_32B_Pure.csv" , index=False)